In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 32391394
paper_name = 'wilcox_austriaco_2020' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/mic-07-129-s01.xlsx', sheet_name='Sheet1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 269 x 1


In [7]:
original_data.head()

,List of SFN Sensitive Mutants
0,ADA2
1,ADE1
2,ADE6
3,ADE8
4,AIM10


In [8]:
original_data['gene'] = original_data['List of SFN Sensitive Mutants'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['gene'] = clean_genename(original_data['gene'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['gene'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [List of SFN Sensitive Mutants, gene, orf]
Index: []


In [12]:
original_data['data'] = -1

In [13]:
original_data.set_index('orf', inplace=True)

In [14]:
original_data = original_data[['data']].copy()

In [15]:
original_data = original_data.groupby(original_data.index).mean()

In [16]:
original_data.shape

(268, 1)

# Prepare the final dataset

In [17]:
data = original_data.copy()

In [18]:
dataset_ids = [21850]
datasets = datasets.reindex(index=dataset_ids)

In [19]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [20]:
data.head()

dataset_id,21850
data_type,value
orf,
YAL023C,-1
YAL044C,-1
YAR015W,-1
YBL032W,-1
YBL046W,-1


## Subset to the genes currently in SGD

In [21]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [22]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,21850
,data_type,value
gene_id,orf,
21,YAL023C,-1
42,YAL044C,-1
67,YAR015W,-1
120,YBL032W,-1
134,YBL046W,-1


# Normalize

In [23]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [24]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [25]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      21850          
data_type       value    valuez
gene_id orf                    
21      YAL023C    -1 -4.122201
42      YAL044C    -1 -4.122201
67      YAR015W    -1 -4.122201
120     YBL032W    -1 -4.122201
134     YBL046W    -1 -4.122201

# Print out

In [26]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [27]:
from IO.save_data_to_db3 import *

In [28]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 32391394...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  1.25it/s]

Updating the data_modified_on field...
